In [73]:
import pandas as pd
import re
import numpy as np

Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

если источник traffic_source равен Yandex или Google, то в source_type ставится organic;

для источников paid и email из России ставим ad;

для источников paid и email не из России ставим other;

все остальные варианты берём из traffic_source без изменений.

In [2]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [22]:
log['source_type'] = log['traffic_source']
log.loc[(log.traffic_source.str.lower() == 'yandex') | (log.traffic_source.str.lower() == 'google'), 'source_type'] = 'organic'
log.loc[(log.traffic_source.str.lower() == 'paid') & (log.region.str.capitalize() == 'Russia'), 'source_type'] = 'ad'
log.loc[(log.traffic_source.str.lower() == 'paid') & (~(log.region.str.capitalize() == 'Russia')), 'source_type'] = 'other'

Задание 2

В файле URLs.txt содержатся URL страниц новостного сайта. 

Вам нужно отфильтровать его по адресам страниц с текстами новостей.

Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. 

Выполните действия:

Прочитайте содержимое файла с датафрейм.

Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [34]:
urls = pd.read_csv('URLs.txt')
urls.loc[urls.url.str.contains(r'\/[0-9]{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 

Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [94]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings1 = ratings.groupby('userId', as_index=False)\
    .agg({'rating': 'count', 'timestamp':lambda x: np.round((x.max() - x.min())/3600, 2)})\
    .rename(columns = {'rating': 'estimates', 'timestamp': 'live_cicl_hours'})
ratings1 = ratings1.loc[ratings1.estimates>100].sort_values(['estimates','live_cicl_hours'], ascending = False)
print(f'Среднее время жизни пользователя, поставившего более 100 оценок равно {float(np.round(ratings1.live_cicl_hours.mean(), 2))} часов')

Среднее время жизни пользователя, поставившего более 100 оценок равно 11133.47 часов


Задание 4

Дана статистика услуг перевозок клиентов компании по типам 

(см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”»

 ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;

аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [95]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [96]:
joined = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
joined

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [97]:
joined_with_address = client_base.merge(joined, on='client_id', how='outer')
joined_with_address

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
